In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go


# Data Extraction

In [2]:
paid = pd.read_csv("./../data/paid_platform.csv")
paid

,Utm Medium,Utm Source,Traffic Week,# Impressions,# Clicks,$ Cost
0,paid-social,linkedin,2023-10-23,4881796,21954,"$169,017.58"
1,paid-social,linkedin,2023-10-16,6335386,15776,"$174,972.20"
2,paid-social,linkedin,2023-10-09,4485024,35009,"$327,466.25"
3,paid-social,linkedin,2023-10-02,4348074,27765,"$272,559.09"
4,paid-social,linkedin,2023-09-25,5962092,31388,"$249,770.65"
...,...,...,...,...,...,...
79,paid-search,bing,2023-07-03,48173,5662,"$28,060.26"
80,paid-search,bing,2023-06-26,52959,5267,"$23,667.38"
81,paid-search,bing,2023-06-19,57243,3996,"$23,881.69"
82,paid-search,bing,2023-06-12,23783,3195,"$18,257.71"


In [3]:
conversion = pd.read_csv("./../data/conversion_data.csv")
conversion

,Touched Date,Utm Medium,Utm Source,Utm Campaign,# Demo Occurred,# Sales Qualified Opportunity,# Closed Won Opportunity
0,2023-10-29,paid-search,google,nam-t1_acq_searchnonbranded_google_search_coun...,2.5,0.0,0.0
1,2023-10-29,paid-listicle,peoplemanagingpeople,ww_acq_prosp_pmp_paidlist_cadpayroll-pmp-__all,2.5,2.5,0.0
2,2023-10-29,paid-social,facebook,nam-t1_consider_calculate_fb_image_all-reengag...,2.4,0.0,0.0
3,2023-10-29,paid-search,google,emea-t1a_acq_searchnonbranded_google_search_pa...,0.0,0.0,0.0
4,2023-10-29,sponsored-newsletter,hrci,ww_engage_download_hrci_sponnewsletter_hr-hrbu...,0.0,0.0,0.0
...,...,...,...,...,...,...,...
11006,2023-06-05,659706244314,149096095629,NaN,1.9,2.5,0.0
11007,2023-06-05,webinar,webinar,ww_acq_prosp_webinar__defaultcamp--__all,0.0,0.0,0.0
11008,2023-06-05,cpc,bing,NaN,0.0,0.0,0.0
11009,2023-06-05,paid-social,facebook,emea-t2_acq_prosp_fb_image_all-cb-__smbmm,3.1,0.0,0.0


# Task 1: Analyzing Sales Funnel Data

## 1) Analyzing Seasonality in Sales:

In [78]:
# Convert to datetime and extract the weekday
conversion['Touched Date'] = pd.to_datetime(conversion['Touched Date'])
conversion['Month'] = conversion['Touched Date'].dt.to_period('M')

### By Monthly Level

In [87]:

 
avg_demo = monthly_data['# Demo Occurred'].mean()
avg_sqo = monthly_data['# Sales Qualified Opportunity'].mean()
avg_cwo = monthly_data['# Closed Won Opportunity'].mean()

 
monthly_data['Demo % Diff'] = ((monthly_data['# Demo Occurred'] / avg_demo) - 1) * 100
monthly_data['SQO % Diff'] = ((monthly_data['# Sales Qualified Opportunity'] / avg_sqo) - 1) * 100
monthly_data['CWO % Diff'] = ((monthly_data['# Closed Won Opportunity'] / avg_cwo) - 1) * 100

 
months = monthly_data['Month'].tolist()
demo_values = monthly_data['# Demo Occurred'].tolist()
sqo_values = monthly_data['# Sales Qualified Opportunity'].tolist()
cwo_values = monthly_data['# Closed Won Opportunity'].tolist()
demo_diff = monthly_data['Demo % Diff'].tolist()
sqo_diff = monthly_data['SQO % Diff'].tolist()
cwo_diff = monthly_data['CWO % Diff'].tolist()

 
fig = go.Figure()  
fig.add_trace(go.Scatter(x=months, y=demo_values,
                         mode='lines+markers',
                         name='# Demo Occurred',
                         line=dict(color='royalblue')))

# average line 
fig.add_trace(go.Scatter(x=months, y=[avg_demo] * len(months),
                         mode='lines',
                         name='Avg # Demo Occurred',
                         line=dict(color='royalblue', dash='dash')))

 
for i, (month, demo_val, demo_d) in enumerate(zip(months, demo_values, demo_diff)):
    fig.add_annotation(x=month, y=demo_val + 240,
                       text=f"{demo_val:.2f} ({demo_d:.2f}% vs avg)",
                       showarrow=False)

fig.add_trace(go.Scatter(x=months, y=sqo_values,
                         mode='lines+markers',
                         name='# Sales Qualified Opportunity',
                         line=dict(color='firebrick')))

# average line  
fig.add_trace(go.Scatter(x=months, y=[avg_sqo] * len(months),
                         mode='lines',
                         name='Avg # Sales Qualified Opportunity',
                         line=dict(color='firebrick', dash='dash')))

 
for i, (month, sqo_val, sqo_d) in enumerate(zip(months, sqo_values, sqo_diff)):
    fig.add_annotation(x=month, y=sqo_val + 240,
                       text=f"{sqo_val:.2f} ({sqo_d:.2f}% vs avg)",
                       showarrow=False)

fig.add_trace(go.Scatter(x=months, y=cwo_values,
                         mode='lines+markers',
                         name='# Closed Won Opportunity',
                         line=dict(color='green')))

 # average line 
fig.add_trace(go.Scatter(x=months, y=[avg_cwo] * len(months),
                         mode='lines',
                         name='Avg # Closed Won Opportunity',
                         line=dict(color='green', dash='dash')))

 
for i, (month, cwo_val, cwo_d) in enumerate(zip(months, cwo_values, cwo_diff)):
    fig.add_annotation(x=month, y=cwo_val + 240,
                       text=f"{cwo_val:.2f} ({cwo_d:.2f}% vs avg)",
                       showarrow=False)
 
fig.update_layout(title='Monthly Trends of Conversion Metrics',
                  xaxis_title='Months',
                  yaxis_title='Metric Values',
                  plot_bgcolor='white',
                  height=600)   
fig.show()


>__Demo Occurrences__: This metric shows good stability (small variability vs average), indicating a consistency on the initial engagement.

>__Sales Qualified Opportunities__: Similar to Demo Occurrences.

>__Closed Won Opportunities__: There's a noticeable decline trend. Specially in the last months. More information should be dig to find out why and where the decline is happening.

## Funnel

In [75]:



# Aggregate data by month
monthly_data = conversion.groupby('Month').agg({
    '# Demo Occurred': 'sum',
    '# Sales Qualified Opportunity': 'sum',
    '# Closed Won Opportunity': 'sum'
}).reset_index()
monthly_data

,Month,# Demo Occurred,# Sales Qualified Opportunity,# Closed Won Opportunity
0,2023-06,3883.8,2242.9,632.5
1,2023-07,4008.2,1953.4,575.8
2,2023-08,4017.5,2122.1,458.5
3,2023-09,3832.6,2070.6,426.1
4,2023-10,4000.6,2143.1,254.0


In [77]:
 

# Prepare data for funnel chart - assuming the funnel follows: # Demo Occurred -> # Sales Qualified Opportunity -> # Closed Won Opportunity
funnel_stages = ['# Demo Occurred', '# Sales Qualified Opportunity', '# Closed Won Opportunity']

# Convert 'Month' column to string type
monthly_data['Month'] = monthly_data['Month'].astype(str)

# Extract unique months
months = monthly_data['Month'].unique().tolist()

fig = go.Figure()
for month_name in months:
    month_data = monthly_data[monthly_data['Month'] == month_name]
    x_values = [month_data[stage].values[0] for stage in funnel_stages]
    fig.add_trace(go.Funnel(
        name=month_name,
        y=funnel_stages,
        x=x_values,
        textinfo="value+percent initial"))

fig.show()


### By  Weekday level

In [18]:
conversion['Weekday'] = conversion['Touched Date'].dt.day_name()
conversion

,Touched Date,Utm Medium,Utm Source,Utm Campaign,# Demo Occurred,# Sales Qualified Opportunity,# Closed Won Opportunity,Weekday
0,2023-10-29,paid-search,google,nam-t1_acq_searchnonbranded_google_search_coun...,2.5,0.0,0.0,Sunday
1,2023-10-29,paid-listicle,peoplemanagingpeople,ww_acq_prosp_pmp_paidlist_cadpayroll-pmp-__all,2.5,2.5,0.0,Sunday
2,2023-10-29,paid-social,facebook,nam-t1_consider_calculate_fb_image_all-reengag...,2.4,0.0,0.0,Sunday
3,2023-10-29,paid-search,google,emea-t1a_acq_searchnonbranded_google_search_pa...,0.0,0.0,0.0,Sunday
4,2023-10-29,sponsored-newsletter,hrci,ww_engage_download_hrci_sponnewsletter_hr-hrbu...,0.0,0.0,0.0,Sunday
...,...,...,...,...,...,...,...,...
11006,2023-06-05,659706244314,149096095629,NaN,1.9,2.5,0.0,Monday
11007,2023-06-05,webinar,webinar,ww_acq_prosp_webinar__defaultcamp--__all,0.0,0.0,0.0,Monday
11008,2023-06-05,cpc,bing,NaN,0.0,0.0,0.0,Monday
11009,2023-06-05,paid-social,facebook,emea-t2_acq_prosp_fb_image_all-cb-__smbmm,3.1,0.0,0.0,Monday


In [13]:
weekly_data = conversion.groupby('Weekday').sum([['# Demo Occurred','# Sales Qualified Opportunity','# Closed Won Opportunity']])\
  .reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

In [17]:

#get max values for each metric
max_demo = weekly_data['# Demo Occurred'].max()
max_sqo = weekly_data['# Sales Qualified Opportunity'].max()
max_cwo = weekly_data['# Closed Won Opportunity'].max()

#get % difference with max
weekly_data['Demo % Diff'] = ((weekly_data['# Demo Occurred'] / max_demo) - 1) * 100
weekly_data['SQO % Diff'] = ((weekly_data['# Sales Qualified Opportunity'] / max_sqo) - 1) * 100
weekly_data['CWO % Diff'] = ((weekly_data['# Closed Won Opportunity'] / max_cwo) - 1) * 100


fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_data.index, y=weekly_data['# Demo Occurred'],
                         mode='lines+markers+text',
                         name='# Demo Occurred',
                         text=[f"{v:.2f} ({d:+.0f}% vs max)" for v, d in zip(weekly_data['# Demo Occurred'], weekly_data['Demo % Diff'])],
                         textposition="top center",
                         marker=dict(color='royalblue')))

fig.add_trace(go.Scatter(x=weekly_data.index, y=weekly_data['# Sales Qualified Opportunity'],
                         mode='lines+markers+text',
                         name='# Sales Qualified Opportunity',
                         text=[f"{v:.2f} ({d:+.0f}% vs max)" for v, d in zip(weekly_data['# Sales Qualified Opportunity'], weekly_data['SQO % Diff'])],
                         textposition="top center",
                         marker=dict(color='firebrick')))

fig.add_trace(go.Scatter(x=weekly_data.index, y=weekly_data['# Closed Won Opportunity'],
                         mode='lines+markers+text',
                         name='# Closed Won Opportunity',
                         text=[f"{v:.2f} ({d:+.0f}% vs max)" for v, d in zip(weekly_data['# Closed Won Opportunity'], weekly_data['CWO % Diff'])],
                         textposition="top center",
                         marker=dict(color='green')))

# Update layout
fig.update_layout(title='Weekly Trends of Conversion Metrics',
                  xaxis_title='Weekday',
                  yaxis_title='Metric Values',
                  plot_bgcolor='white',)

fig.show()

## Autocorrolation

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(df['your_column_name'])
plt.title('Autocorrelation Function')
plt.show()

plot_pacf(df['your_column_name'])
plt.title('Partial Autocorrelation Function')
plt.show()
